# Classification models - Supervised Discretization
## Dataset: Satimage
<br>
Updated by: Sam <br>
Update at: 13/10/2022 <br>
--- 
Discretization methods: DecisionTree (DT) <br>
Classification models: CNB, ID3, KNN-VDM<br>
---
NOTE:

!!! NB. There are no examples with class 6 in this dataset-they have all been removed because of doubts about the 
	validity of this class. <br>
!!! NB. DO NOT USE CROSS-VALIDATION WITH THIS DATASET !!!
- Just train and test only once with the above training and test sets.<br>
- The data is given in random order and certain lines of data have been removed so you cannot reconstruct the original image from this dataset. <br>

Long time for computation of Knn-VDM (Run this part last), only finish training <br>
Cannot finish bias-variance decomposition for KNN-VDM models => no metrics for these models

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, recall_score, precision_score
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import six
import sys
sys.modules['sklearn.externals.six'] = six
import id3
from id3 import Id3Estimator
from id3 import export_graphviz
import wittgenstein as lw
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from pandas import read_csv
from pandas import set_option
import numpy as np
from numpy import arange
## EDA
from collections import Counter

# Knn-VDM 3
from vdm3 import ValueDifferenceMetric
from sklearn.neighbors import KNeighborsClassifier

# Pre-processing
from sklearn.preprocessing import OrdinalEncoder
# Cross validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score # 1 metric
from sklearn.model_selection import cross_validate # more than 1 metric
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [3]:
#read in data from ChiMerge discretization
satimage_m2 = pd.read_csv('DT_small_discretized_satimage.csv')
satimage_m3 = pd.read_csv('DT_medium_discretized_satimage.csv')
satimage_m4 = pd.read_csv('DT_large_discretized_satimage.csv')
satimage_m5 = pd.read_csv('DT_verylarge_discretized_satimage.csv')

In [6]:
num_list_m2 = satimage_m2.columns.drop('class')
num_list_m3 = satimage_m3.columns.drop('class')
num_list_m4 = satimage_m4.columns.drop('class')
num_list_m5 = satimage_m5.columns.drop('class')

In [12]:
train= satimage_m2.head(4435)
test= satimage_m2.tail(2000)
train
test

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A28,A29,A30,A31,A32,A33,A34,A35,A36,class
4435,0,0,2,2,0,0,3,2,0,0,...,1,0,0,3,2,0,0,3,2,2
4436,2,2,1,1,2,2,1,0,2,2,...,2,1,1,3,2,0,0,3,2,5
4437,2,1,2,2,2,2,3,2,2,2,...,1,2,2,3,2,2,2,3,2,4
4438,2,2,0,1,2,2,2,0,2,2,...,2,2,2,1,1,2,2,2,0,3
4439,1,2,2,2,2,1,2,2,2,1,...,1,0,0,3,2,0,1,3,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430,2,2,0,0,2,2,0,0,2,2,...,1,2,2,2,1,2,2,2,0,5
6431,1,1,2,2,2,2,2,2,2,2,...,1,1,2,2,2,0,1,3,2,3
6432,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,1
6433,2,2,0,0,2,2,0,0,2,2,...,0,2,2,0,0,2,2,0,1,4


# 1. Categorical Naive Bayes

## 1.1 CNB, DT, max_depth = 2

In [13]:
# make test & train split
train= satimage_m2.head(4435)
test= satimage_m2.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']

In [14]:
# SAM
# Count number of categories
n_categories = satimage_m2.drop('class', axis=1).nunique()

In [16]:
import time
start = time.time() # For measuring time execution

cnb = CategoricalNB(min_categories = n_categories) # Update 13/10
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 526
Accuracy: 0.74
Recall score :  0.737
Precision score :  0.737
F1 score :  0.737
Classification report:
              precision    recall  f1-score   support

           0       0.88      0.78      0.83       489
           1       0.98      0.91      0.95       202
           2       0.88      0.92      0.90       424
           3       0.42      0.59      0.50       214
           4       0.35      0.27      0.30       227
           5       0.71      0.74      0.72       444

    accuracy                           0.74      2000
   macro avg       0.71      0.70      0.70      2000
weighted avg       0.74      0.74      0.74      2000

Computation time:
0.03257870674133301


In [17]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.263
Average bias: 0.263
Average variance: 0.018
Sklearn 0-1 loss: 0.263


## 1.2 CNB, DT, max_depth = 3

In [18]:
# make test & train split
train= satimage_m3.head(4435)
test= satimage_m3.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']

In [19]:
# SAM
# Count number of categories
n_categories = satimage_m3.drop('class', axis=1).nunique()

In [20]:
import time
start = time.time() # For measuring time execution

cnb = CategoricalNB(min_categories = n_categories) # Update 13/10
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 352
Accuracy: 0.82
Recall score :  0.824
Precision score :  0.824
F1 score :  0.824
Classification report:
              precision    recall  f1-score   support

           0       0.91      0.84      0.87       489
           1       0.98      0.91      0.95       202
           2       0.89      0.93      0.91       424
           3       0.54      0.65      0.59       214
           4       0.65      0.76      0.70       227
           5       0.89      0.79      0.83       444

    accuracy                           0.82      2000
   macro avg       0.81      0.81      0.81      2000
weighted avg       0.84      0.82      0.83      2000

Computation time:
0.030998706817626953


In [21]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.176
Average bias: 0.176
Average variance: 0.012
Sklearn 0-1 loss: 0.176


## 1.3 CNB, DT, max_depth = 4

In [22]:
train= satimage_m4.head(4435)
test= satimage_m4.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']

In [23]:
# SAM
# Count number of categories
n_categories = satimage_m4.drop('class', axis=1).nunique()

In [24]:
import time
start = time.time() # For measuring time execution

cnb = CategoricalNB(min_categories = n_categories)
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 353
Accuracy: 0.82
Recall score :  0.8235
Precision score :  0.8235
F1 score :  0.8235
Classification report:
              precision    recall  f1-score   support

           0       0.93      0.82      0.87       489
           1       0.98      0.91      0.95       202
           2       0.88      0.92      0.90       424
           3       0.55      0.70      0.62       214
           4       0.65      0.77      0.70       227
           5       0.88      0.78      0.83       444

    accuracy                           0.82      2000
   macro avg       0.81      0.82      0.81      2000
weighted avg       0.84      0.82      0.83      2000

Computation time:
0.032097816467285156


In [25]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.176
Average bias: 0.176
Average variance: 0.012
Sklearn 0-1 loss: 0.176


## 1.4 CNB, DT, max_depth = 5

In [26]:
train= satimage_m5.head(4435)
test= satimage_m5.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']

In [27]:
# SAM
# Count number of categories
n_categories = satimage_m5.drop('class', axis=1).nunique()

In [28]:
import time
start = time.time() # For measuring time execution

cnb = CategoricalNB(min_categories = n_categories) # Update 13/10
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 347
Accuracy: 0.83
Recall score :  0.8265
Precision score :  0.8265
F1 score :  0.8265
Classification report:
              precision    recall  f1-score   support

           0       0.93      0.83      0.88       489
           1       0.99      0.92      0.95       202
           2       0.88      0.93      0.90       424
           3       0.55      0.70      0.61       214
           4       0.67      0.76      0.71       227
           5       0.88      0.78      0.83       444

    accuracy                           0.83      2000
   macro avg       0.81      0.82      0.81      2000
weighted avg       0.84      0.83      0.83      2000

0.028133869171142578


In [29]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.173
Average bias: 0.172
Average variance: 0.015
Sklearn 0-1 loss: 0.173


# 2. Decision Tree ID3

## 2.1 ID3 model, DT, max_depth = 2

### Debug version

In [24]:
train= satimage_m2.head(4435)
test= satimage_m2.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']

In [25]:
import time
start = time.time() # For measuring time execution

estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)

y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Computation time:
1.148970603942871


In [26]:
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))

Accuracy: 0.75
Recall score :  0.752
Precision score :  0.752
F1 score :  0.752
Classification report:
              precision    recall  f1-score   support

           0       0.84      0.86      0.85       489
           1       0.95      0.94      0.94       202
           2       0.87      0.89      0.88       424
           3       0.48      0.45      0.47       214
           4       0.44      0.38      0.41       227
           5       0.71      0.75      0.73       444

    accuracy                           0.75      2000
   macro avg       0.72      0.71      0.71      2000
weighted avg       0.75      0.75      0.75      2000



In [27]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

KeyboardInterrupt: 

### Complete version

In [ ]:
#make splits
train= satimage_m2.head(4435)
test= satimage_m2.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

In [ ]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

## 2.2 ID3 model, DT, max_depth = 3

In [ ]:
#make splits
train= satimage_m3.head(4435)
test= satimage_m3.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

In [ ]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

## 2.3 ID3 model, DT, max_depth = 4

In [ ]:
#make splits
train= satimage_m4.head(4435)
test= satimage_m4.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

In [ ]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

## 2.4 ID3 model, DT, max_depth = 5

In [ ]:
#make splits
train= satimage_m5.head(4435)
test= satimage_m5.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

In [ ]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

# 3. KNN classification

In [ ]:
#read in data from ChiMerge discretization
satimage_m2 = pd.read_csv('ChiM_discretized_6Intervals_satimage.csv')
satimage_m3 = pd.read_csv('ChiM_discretized_8Intervals_satimage.csv')
satimage_m4 = pd.read_csv('ChiM_discretized_10Intervals_satimage.csv')
satimage_m5 = pd.read_csv('ChiM_discretized_15Intervals_satimage.csv')

In [ ]:
# # Complete code for data preperation
# # Read data
# #df_ewd1 = satimage_m2
# disc = 'EWD'
# k = 4

# # df_ewd1.info()
# # data = df_ewd1.values
# # data.shape

# # train= satimage_m2.head(4435)
# # test= satimage_m2.tail(2000)

# train= satimage_m2.head(3000)
# test= satimage_m2.tail(1000)

# data_train = train.values
# data_train.shape
# data_test = test.values
# data_test.shape
# features = satimage_m2.drop('class', axis = 1).columns


# # separate the data into X and y
# x_train = data_train[:, : len(features)]
# y_train = data_train[:,-1]
# x_test = data_test[:, : len(features)]
# y_test = data_test[:,-1]

# #print(X.shape, Y.shape)

# # Split train test
# #x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30) 

# # Check representation of class
# #print('Class representation - original: ', Counter(Y)) 
# print('Class representation - training data: ', Counter(y_train)) 
# print('Class representation - testing data: ', Counter(y_test)) 

In [ ]:
# # Knn-VDM complete code
# import time
# start = time.time() # For measuring time execution

# # specific the continuous columns index if any
# vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
# vdm.fit()
# # Knn model, n_neigbour = 3, metrics = vdm
# knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# # Fit model
# knn_vdm.fit(x_train, y_train)
# # Testing
# y_pred_knn = knn_vdm.predict(x_test)
# knn_vdm.classes_
# print(classification_report(y_test, y_pred_knn))

# end = time.time()
# print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

In [ ]:
# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         knn_vdm, x_train, y_train, x_test, y_test, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

## 3.1 KNN-VDM model, DT, max_depth = 2

In [72]:
# Complete code for data preperation
# Read data
#df_ewd1 = satimage_m2
disc = 'EWD'
k = 4

# df_ewd1.info()
# data = df_ewd1.values
# data.shape

# train= satimage_m2.head(4435)
# test= satimage_m2.tail(2000)

train= satimage_m2.head(500)
test= satimage_m2.tail(100)

data_train = train.values
data_train.shape
data_test = test.values
data_test.shape
features = satimage_m2.drop('class', axis = 1).columns


# separate the data into X and y
x_train = data_train[:, : len(features)]
y_train = data_train[:,-1]
x_test = data_test[:, : len(features)]
y_test = data_test[:,-1]

#print(X.shape, Y.shape)

# Split train test
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30) 

# Check representation of class
#print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - training data:  Counter({5: 137, 2: 111, 0: 108, 1: 53, 3: 46, 4: 45})
Class representation - testing data:  Counter({2: 25, 0: 22, 1: 16, 5: 16, 3: 13, 4: 8})


In [73]:
for i in satimage_m2.columns:
    #print('Interval numbers for '+i)
    print(len(Counter(train[i])))

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
6


In [74]:
for i in train.columns:
    #print('Interval numbers for '+i)
    print(len(Counter(train[i])))

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
6


In [75]:
for i in test.columns:
    #print('Interval numbers for '+i)
    print(len(Counter(train[i])))

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
6


In [76]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       0.78      0.95      0.86        22
           1       1.00      0.69      0.81        16
           2       0.80      0.96      0.87        25
           3       0.60      0.23      0.33        13
           4       1.00      0.38      0.55         8
           5       0.62      0.94      0.75        16

    accuracy                           0.77       100
   macro avg       0.80      0.69      0.70       100
weighted avg       0.79      0.77      0.74       100

Time for training model Knn-VDM: 88.16726541519165.


In [31]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, x_train, y_train, x_test, y_test, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

KeyboardInterrupt: 

## 3.2 KNN-VDM model, DT, max_depth = 3

In [77]:
# Complete code for data preperation
# Read data
#df_ewd1 = satimage_m2
disc = 'EWD'
k = 4

# df_ewd1.info()
# data = df_ewd1.values
# data.shape

# train= satimage_m2.head(4435)
# test= satimage_m2.tail(2000)

train= satimage_m2.head(500)
test= satimage_m2.tail(100)

data_train = train.values
data_train.shape
data_test = test.values
data_test.shape
features = satimage_m2.drop('class', axis = 1).columns


# separate the data into X and y
x_train = data_train[:, : len(features)]
y_train = data_train[:,-1]
x_test = data_test[:, : len(features)]
y_test = data_test[:,-1]

#print(X.shape, Y.shape)

# Split train test
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30) 

# Check representation of class
#print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - training data:  Counter({5: 137, 2: 111, 0: 108, 1: 53, 3: 46, 4: 45})
Class representation - testing data:  Counter({2: 25, 0: 22, 1: 16, 5: 16, 3: 13, 4: 8})


In [78]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       0.78      0.95      0.86        22
           1       1.00      0.69      0.81        16
           2       0.80      0.96      0.87        25
           3       0.60      0.23      0.33        13
           4       1.00      0.38      0.55         8
           5       0.62      0.94      0.75        16

    accuracy                           0.77       100
   macro avg       0.80      0.69      0.70       100
weighted avg       0.79      0.77      0.74       100

Time for training model Knn-VDM: 92.01916193962097.


In [ ]:
# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         knn_vdm, X_train.values, y_train.values, X_test.values, y_test.values, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

## 3.3 KNN-VDM model, DT, max_depth = 4

In [84]:
# Complete code for data preperation
# Read data
#df_ewd1 = satimage_m2
disc = 'EWD'
k = 4

# df_ewd1.info()
# data = df_ewd1.values
# data.shape

# train= satimage_m2.head(4435)
# test= satimage_m2.tail(2000)

train= satimage_m4.head(1000)
test= satimage_m4.tail(500)

data_train = train.values
data_train.shape
data_test = test.values
data_test.shape
features = satimage_m2.drop('class', axis = 1).columns


# separate the data into X and y
x_train = data_train[:, : len(features)]
y_train = data_train[:,-1]
x_test = data_test[:, : len(features)]
y_test = data_test[:,-1]

#print(X.shape, Y.shape)

# Split train test
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30) 

# Check representation of class
#print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - training data:  Counter({5: 260, 2: 221, 0: 214, 1: 109, 4: 106, 3: 90})
Class representation - testing data:  Counter({0: 132, 2: 115, 5: 112, 1: 49, 4: 46, 3: 46})


In [85]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       132
           1       0.96      0.96      0.96        49
           2       0.84      0.90      0.87       115
           3       0.51      0.48      0.49        46
           4       0.76      0.57      0.65        46
           5       0.83      0.81      0.82       112

    accuracy                           0.84       500
   macro avg       0.80      0.78      0.79       500
weighted avg       0.83      0.84      0.83       500

Time for training model Knn-VDM: 948.4903318881989.


In [ ]:
# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         knn_vdm, X_train.values, y_train.values, X_test.values, y_test.values, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

## 3.4 KNN-VDM model, DT, max_depth = 5

In [86]:
# Complete code for data preperation
# Read data
#df_ewd1 = satimage_m2
disc = 'EWD'
k = 4

# df_ewd1.info()
# data = df_ewd1.values
# data.shape

# train= satimage_m2.head(4435)
# test= satimage_m2.tail(2000)

train= satimage_m4.head(1000)
test= satimage_m4.tail(500)

data_train = train.values
data_train.shape
data_test = test.values
data_test.shape
features = satimage_m2.drop('class', axis = 1).columns


# separate the data into X and y
x_train = data_train[:, : len(features)]
y_train = data_train[:,-1]
x_test = data_test[:, : len(features)]
y_test = data_test[:,-1]

#print(X.shape, Y.shape)

# Split train test
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30) 

# Check representation of class
#print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - training data:  Counter({5: 260, 2: 221, 0: 214, 1: 109, 4: 106, 3: 90})
Class representation - testing data:  Counter({0: 132, 2: 115, 5: 112, 1: 49, 4: 46, 3: 46})


In [87]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       132
           1       0.96      0.96      0.96        49
           2       0.84      0.90      0.87       115
           3       0.51      0.48      0.49        46
           4       0.76      0.57      0.65        46
           5       0.83      0.81      0.82       112

    accuracy                           0.84       500
   macro avg       0.80      0.78      0.79       500
weighted avg       0.83      0.84      0.83       500

Time for training model Knn-VDM: 1114.753978252411.


In [1]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

NameError: name 'bias_variance_decomp' is not defined